In [ ]:
!pip install accelerate -U

In [ ]:
!pip install einops

In [ ]:
!pip install transformers==4.29

In [ ]:
from transformers import AutoTokenizer, BertForSequenceClassification,  Trainer, TrainingArguments

tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNABERT-2-117M",trust_remote_code= True)
model = BertForSequenceClassification.from_pretrained("zhihan1996/DNABERT-2-117M",trust_remote_code=True, num_labels=9)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Some weights of the model checkpoint at zhihan1996/DNABERT-2-117M were not used when initializing BertForSequenceClassification: ['bert.encoder.layer.9.mlp.gated_layers.weight', 'bert.encoder.layer.9.mlp.wo.weight', 'bert.encoder.layer.1.mlp.gated_layers.weight', 'bert.encoder.layer.6.mlp.gated_layers.weight', 'bert.encoder.layer.6.mlp.layernorm.weight', 'bert.encoder.layer.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
train=pd.read_csv("drive/My Drive/colabDrive/GUE/GUE/virus/covid/train.csv")
train.head()

,sequence,label
0,TGTTCTTTACCAACCACCACAAATCTCTATCACCTCAGCTGTTTTG...,7
1,ACATTGCCAAAAGGCTTCTACGCAGAAGGGAGCAGAGGCGGCAGTC...,3
2,GCTCAAACTGGAATTGCCGTTTTAGATATGTGTGCTTCATTAAAAG...,1
3,CGGTGCTTTACTTACAAAGTCCTCAGAATACAAAGGTCCTATTACG...,4
4,CCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTT...,0


In [ ]:
test=pd.read_csv("drive/My Drive/colabDrive/GUE/GUE/virus/covid/test.csv")
test.head()

,sequence,label
0,ACATTAATGGCAATCTTCATCCAGATTCTGCCACTCTTGTTAGTGA...,2
1,GGTGCAGGTATATGCGCTAGTTATCAGACTCAGACTAATTCTCATC...,0
2,GCACCATATGGAGTTACCAACTGGAGTTCATGCTGGCACAGACTTA...,8
3,TAATGGTGTTTCCTTTAGTACTTTTGAAGAAGCTGCGCTGTGCACC...,0
4,TTGATGAAGGTAATTGTGATACATTAAAAGAAATACTTGTCACATA...,8


In [ ]:
dev=pd.read_csv("drive/My Drive/colabDrive/GUE/GUE/virus/covid/dev.csv")
dev.head()

,sequence,label
0,TATGTCTGAAGCAAAATGTTGGACTGAGACTGACCTTACTAAAGGA...,3
1,ATGGCACACACTGGTTTGTAACACAAAGGAATTTTTATGAACCACA...,6
2,ACCATTTCATCCTCTAGCTGATAACAAATTTGCACTGACTTGCTTT...,8
3,TTTGTTGACAGGCAAACAGCACAAGCAGCTGGTACGGACACAACTA...,2
4,TTAGATTTCCTAATATTACAAACTTGTGCCCTTTTGGTGAAGTTTT...,4


In [ ]:
text = pd.concat([train, test, dev], ignore_index=True)
text.head()

,sequence,label
0,TGTTCTTTACCAACCACCACAAATCTCTATCACCTCAGCTGTTTTG...,7
1,ACATTGCCAAAAGGCTTCTACGCAGAAGGGAGCAGAGGCGGCAGTC...,3
2,GCTCAAACTGGAATTGCCGTTTTAGATATGTGTGCTTCATTAAAAG...,1
3,CGGTGCTTTACTTACAAAGTCCTCAGAATACAAAGGTCCTATTACG...,4
4,CCAACCAACTTTCGATCTCTTGTAGATCTGTTCTCTAAACGAACTT...,0


In [ ]:
!pip install torch
import torch
from torch.utils.data import Dataset, DataLoader,random_split

In [ ]:
class DNADataset(Dataset):
    def __init__(self, sequence, label, tokenizer, max_length):
        self.sequence = sequence
        self.label = label
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.sequence)

    def __getitem__(self, idx):
        sequence = self.sequence[idx]
        label = self.label[idx]
        encoding = self.tokenizer.encode_plus(
            sequence,
            add_special_tokens=True,
            max_length=self.max_length,
            return_token_type_ids=False,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='pt',
        )
        return {
            'sequence': sequence,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

def create_data_loader(df, tokenizer, max_length, batch_size):
    ds = DNADataset(
        sequence=df.sequence.to_numpy(),
        label=df.label.to_numpy(),
        tokenizer=tokenizer,
        max_length=max_length
    )
    return DataLoader(
        ds,
        batch_size=batch_size
    )

In [ ]:
from sklearn.metrics import accuracy_score
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return {'accuracy': accuracy_score(labels,predictions)}

In [ ]:
MAX_LEN = 400
dataset = DNADataset(dev.sequence, dev.label, tokenizer, MAX_LEN)

# Split the dataset into training and validation sets
train_size = int(0.2 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])


# Define the training arguments
training_args = TrainingArguments(
    output_dir='/drive/MyDrive/colabdrive/result',
    num_train_epochs=1,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    warmup_steps=50,
    weight_decay=0.01,
    logging_dir='/drive/MyDrive/colabdrive/log',
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
# Save the trained model
model.save_pretrained('/drive/MyDrive/colabdrive/model')
# Load the saved model
loaded_model = BertForSequenceClassification.from_pretrained('/drive/MyDrive/colabdrive/model')

In [ ]:
eval_results=trainer.evaluate()
print(f'Evaluation Accuracy: {eval_results["eval_accuracy"]}')

In [ ]:
# Use the loaded model for prediction
sequence = 'TATGTCTGAAGCAAAATGTTGGACTGAGACTGACCTTACTAAAGGACCTCATGAATTTTGCTCTCAACATACAATGCTAGTTAAACAGGGTGATGATTATGTGTACCTTCCTTACCCAGATCCATCAAGAATCCTAGGGGCCGGCTGTTTTGTAGATGATATCGTAAAAACAGATGGTACACTTATGATTGAACGGTTCGTGTCTTTAGCTATAGATGCTTACCCACTTACTAAACATCCTAATCAGGAGTATGCTGATGTCTTTCATTTGTACTTACAATACATAAGAAAGCTACATGATGAGTTAACAGGACACATGTTAGACATGTATTCTGTTATGCTTACTAATGATAACACTTCAAGGTATTGGGAACCTGAGTTTTATGAGGCTATGTACACACCGCATACAGTCTTACAGGCTGTTGGGGCTTGTGTTCTTTGCAATTCACAGACTTCATTAAGATGTGGTGCTTGCATACGTAGACCATTCTTATGTTGTAAATGCTGTTACGACCATGTCATATCAACATCACATAAATTAGTCTTGTCTGTTAATCCGTATGTTTGCAATGCTCCAGGTTGTGATGTCACAGATGTGACTCAACTTTACTTAGGAGGTATGAGCTATTATTGTAAATCACATAAACCACCCATTAGTTTTCCATTGTGTGCTAATGGACAAGTTTTTGGTTTATATAAAAATACATGTGTTGGTAGCGATAATGTTACTGACTTTAATGCAATTGCAACATGTGACTGGACAAATGCTGGTGATTACATTTTAGCTAACACCTGTACTGAAAGACTCAAGCTTTTTGCAGCAGAAACGCTCAAAGCTACTGAGGAGACATTTAAACTGTCTTATGGTATTGCTACTGTACGTGAAGTGCTGTCTGACAGAGAATTACATCTTTCATGGGAAGTTGGTAAACCTAGACCACCACTTAACCGAAATTATGTCTTTACTGGTTATCGTGTAACTAAAAACAGTAAAGTACA'
encoding = tokenizer.encode_plus(
    sequence,
    add_special_tokens=True,
    max_length=MAX_LEN,
    return_token_type_ids=False,
    padding='max_length',
    return_attention_mask=True,
    return_tensors='pt',
)
output = loaded_model(**encoding)
prediction = output.logits.argmax().item()
print(f'Prediction for sequence "{sequence}": {prediction}')
